In [1]:
import tools

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
print(os.getenv("LANGSMITH_ENDPOINT"))

https://api.smith.langchain.com


In [3]:
import os

import os
from langchain.chat_models import init_chat_model



llm = init_chat_model(
    "azure_openai:gpt-4.1-mini",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version="2024-12-01-preview"
)

agent_tools = [tools.get_all_plants, tools.get_bloom_data_whole]
llm_with_tools = llm.bind_tools(agent_tools)

In [4]:
from utils import make_specialist

plants = tools.get_all_plants().split(" ")
print(plants)
#create specialists
specialists = {plant: make_specialist(plant, llm) for plant in plants}
[specialist.name for specialist in specialists.values()]

['apfel', 'birke', 'erle', 'hasel', 'winterraps', 'winterroggen']


['apfel_agent',
 'birke_agent',
 'erle_agent',
 'hasel_agent',
 'winterraps_agent',
 'winterroggen_agent']

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage
import json
from utils import get_tokens
#base_prompt = 'Antworte in JSON und folge dem Schema: { "type": "object", "properties": { "response": { "type": "boolean" }, "reasoning": { "type": "string" } }, "required": [ "response", "reasoning" ] }'
with open("datasets/dataset_24.json", "r") as file:
    dataset = json.load(file)


responses = []
for agent in specialists.values():
    for data in dataset:
        sentence = data["sentence"]
        prompt = [HumanMessage(content=sentence)]
        print(f"Processing: {sentence}")
        messages = agent.invoke({"messages": prompt})
        for m in messages['messages']:
            m.pretty_print()
        response = messages["messages"][-1].content
        cleaned_response = response.strip("```json").strip("```").strip()
        response_json = json.loads(cleaned_response)
        tokens = get_tokens(messages)
        responses.append({
            "id": data["id"],
            "plant": data["plant"],
            "city": data["city"],
            "date": data["date"],
            "sentence": sentence,
            "response": response_json["response"],
            "reasoning": response_json["reasoning"],
            "tokens": tokens
        })



    with open(f"results/{agent.name}_24.json", "w") as file:
        json.dump(responses, file, indent=4)

Processing: Heute, am 11.04.2021, befinden wir uns in Berlin. Ist die Apfel gerade am Blühen?
Tool invoked with plant=apfel, year=2021, type=beginn
Tool invoked with plant=apfel, year=2021, type=ende
================================ Human Message =================================

Heute, am 11.04.2021, befinden wir uns in Berlin. Ist die Apfel gerade am Blühen?
================================== Ai Message ==================================
Name: apfel_agent
Tool Calls:
  get_bloom_data_apfel (call_IYj43mMgfBMApeS56woBF4f2)
 Call ID: call_IYj43mMgfBMApeS56woBF4f2
  Args:
    year: 2021
================================= Tool Message =================================
Name: get_bloom_data_apfel

{"begin_data": {"Jahr": {"4": 2021}, "Gebietsmittel": {"4": "01.05. "}, "Abweichung vom Mittel": {"4": "5 Tage"}}, "ende_data": {"Jahr": {"4": 2021}, "Mittel 78 %": {"4": "22.05. "}, "Abweichung 78 %": {"4": "9 Tage "}, "Mittel 100 %": {"4": "26.05. "}, "Abweichung 100 %": {"4": "10 Tage"}}, "info